In [1]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)

In [2]:
from src.models.byol.BYOL import BYOL
from src.scripts.etl_process.ETLProcessor import ETLProcessor
import torch

d:\studia\priv\Inpainting\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
input_dim = 3
hidden_dim = 128
residual_hiddens = 64
num_residual_layers = 2
mlp_hidden_dim = 256
device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
byol = BYOL(
    input_dim=input_dim,
    hidden_dim=hidden_dim,
    num_residual_layers=num_residual_layers,
    residual_hiddens=residual_hiddens,
    device=device,
)

In [5]:
byol

BYOL(
  (online_encoder): Encoder(
    (_conv_1): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (_conv_2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (_conv_3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (_residual_stack): ResidualStack(
      (_layers): ModuleList(
        (0-1): 2 x ResidualBlock(
          (_block): Sequential(
            (0): ReLU(inplace=True)
            (1): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (2): ReLU(inplace=True)
            (3): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          )
        )
      )
    )
  )
  (online_projector): MLP(
    (net): Sequential(
      (0): Linear(in_features=32768, out_features=512, bias=True)
      (1): ReLU(inplace=True)
      (2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): Linear(in_features=512, out_features=512, bias=Tru

In [6]:
etl = ETLProcessor(
    kaggle_dataset="mahmudulhaqueshawon/cat-image",
    raw_dir="data/raw_data",
    split_dir="data/data_splits",
)
train_loader, val_loader, test_loader = etl.process()

[INFO] Dataset 'mahmudulhaqueshawon/cat-image' already exists at data/raw_data, skipping copy.


In [7]:
from tqdm import tqdm
import torch
import torch.optim as optim


def fit_byol(
    model: BYOL,
    optimiser: optim.Optimizer,
    dataloader: torch.utils.data.DataLoader,
    epochs: int,
    device: str = "cuda",
    print_metrics: bool = True,
):
    model.to(device)

    for epoch in range(epochs):
        model.train()
        epoch_loss = 0.0
        num_batches = 0

        for x_batch, _ in tqdm(dataloader, desc=f"Epoch {epoch}"):
            x_batch = x_batch.to(device)

            q, z = model(x_batch)

            loss = model.byol_loss(q, z)

            optimiser.zero_grad()
            loss.backward()
            optimiser.step()

            model.update_target_network()

            epoch_loss += loss.item()
            num_batches += 1

        avg_loss = epoch_loss / num_batches

        if print_metrics:
            print(f"Epoch {epoch}: BYOL training loss = {avg_loss:.8f}")

In [8]:
byol = byol.to(device)
fit_byol(
    model=byol,
    optimiser=optim.Adam(byol.parameters(), lr=1e-4),
    dataloader=train_loader,
    epochs=5,
    device=device,
)

Epoch 0:   0%|          | 0/87 [00:00<?, ?it/s]

v min/max: -1.000/1.000, v_prim min/max: -1.000/0.495
q1 0.03719356283545494 0.04875040054321289
q2 0.03850143775343895 0.05001530796289444
z1 0.03850143775343895 0.05001530796289444
z2 0.03719356283545494 0.04875040054321289


Epoch 0:   1%|          | 1/87 [00:14<20:19, 14.19s/it]

v min/max: -1.000/0.615, v_prim min/max: -1.000/1.000
q1 0.03611569106578827 0.04754255339503288
q2 0.03935832530260086 0.05098680779337883
z1 0.0392623171210289 0.051030777394771576
z2 0.036034002900123596 0.04757463186979294


Epoch 0:   2%|▏         | 2/87 [00:15<09:40,  6.83s/it]

...

v min/max: -1.000/1.000, v_prim min/max: -1.000/1.000
q1 nan nan
q2 nan nan
z1 nan nan
z2 nan nan


Epoch 4:  40%|████      | 35/87 [01:10<01:29,  1.73s/it]

v min/max: -1.000/1.000, v_prim min/max: -1.000/1.000
q1 nan nan
q2 nan nan


Epoch 4:  40%|████      | 35/87 [01:11<01:46,  2.06s/it]


KeyboardInterrupt: 